In [7]:
%%writefile secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: seldon-init-container-secret
type: Opaque
stringData:
  RCLONE_CONFIG_S3_TYPE: s3
  RCLONE_CONFIG_S3_PROVIDER: minio
  RCLONE_CONFIG_S3_ENV_AUTH: "false"
  RCLONE_CONFIG_S3_ACCESS_KEY_ID: minioadmin
  RCLONE_CONFIG_S3_SECRET_ACCESS_KEY: minioadmin
  RCLONE_CONFIG_S3_ENDPOINT: http://minio.minio-system.svc.cluster.local:9000

Overwriting secret.yaml


In [12]:
%%writefile seldon-triton-yolo.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: image-models-yolo
spec:
  name: default
  predictors:
  - graph:
      implementation: TRITON_SERVER
      logger:
        mode: all
      modelUri: s3://separate-servers/separate-servers/yolo
      envSecretRefName: seldon-init-container-secret
      name: image-models-resnet # This should have the same name as the model inside
      type: MODEL
    name: default
    replicas: 1
  protocol: kfserving

Overwriting seldon-triton-yolo.yaml


In [14]:
!kubectl apply -f secret.yaml -n default
!kubectl apply -f seldon-triton-yolo.yaml -n default

secret/seldon-init-container-secret configured
seldondeployment.machinelearning.seldon.io/image-models-yolo created
